In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import time
import pandas as pd
import numpy as np
import requests

import re
from bs4 import BeautifulSoup

os.chdir(r"/content/drive/My Drive/Colab Notebooks/t-brain_2020_NLP_team_share_folder")
# os.chdir(r"/content/drive/My Drive/Colab Notebooks/2020_玉山夏季賽_NLP應用挑戰賽/t-brain_2020_NLP_team_share_folder")

In [ ]:
news_urls = pd.read_csv('raw_data/train_data_custom_0628.csv')

## 內文前處理

### 去除英文字、標點符號、特殊符號

In [ ]:
# 多加全形轉半形
def strQ2B(ustring):
    if isinstance(ustring, str) :
      """把字串全形轉半形"""
      ss = []
      for s in ustring:
          rstring = ""
          for uchar in s:
              inside_code = ord(uchar)
              if inside_code == 12288:  # 全形空格直接轉換
                  inside_code = 32
              elif (inside_code >= 65281 and inside_code <= 65374):  # 全形字元（除空格）根據關係轉化
                  inside_code -= 65248
              rstring += chr(inside_code)
          ss.append(rstring)
      return ''.join(ss)
    else:
      return unstring

news_urls["content"] = news_urls["content"].apply(lambda x: re.sub("[▲◆▪【】*';％%※★<>#〈〉' '_|｜()（）－-]", "", str(strQ2B(x)))) # !！？?。
news_urls["content"] = news_urls["content"].apply(lambda x: re.sub("[!！？?]", "。", str(x)))

## 整理成Training Data格式

In [ ]:
def split(word):
    return [char for char in word]

start_idx = []
end_idx = []
name_list = []
NRE_list = []

for sun_name in truth:
    # sun_name = truth[0]
    sub_start = [m.start() for m in re.finditer(sun_name, input_context)]
    sub_end = list(np.asarray(sub_start) + len(sun_name))
    NRE_list += np.repeat('PERSON', len(sub_start)).tolist()
    name_list += np.repeat(sun_name, len(sub_start)).tolist()
    start_idx += sub_start
    end_idx += sub_end
entity_sentence_list = [set(zip(start_idx, end_idx, NRE_list, name_list))]

review_PER = []
for entity in sorted(entity_sentence_list[0]):
    print(entity)
    if entity[2] == 'PERSON':
        review_PER.append(entity[3])

In [ ]:
import re
# by句子去猜，用句號分隔
from itertools import groupby
from itertools import islice 

def split(word):
    return [char for char in word]

def BIO_encode(content, label):
    """
        content: str
        label: list of names
    """

    if str(label) == "['']": 

      Context_list, IBO_list = split(content)+[''], np.repeat('O', len(content)+1).tolist()

    else:

      start_idx = []
      end_idx = []
      name_list = []
      NER_list = []

      for name in label:
          # name = truth[0]
          name_start = [m.start() for m in re.finditer(name, content)]
          name_end = list(np.asarray(name_start) + len(name))

          # 紀錄entity & 名字名稱
          NER_list += np.repeat('PERSON', len(name_start)).tolist()
          name_list += np.repeat(name, len(name_start)).tolist()
          
          # 名字在文章中的start, end
          start_idx += name_start
          end_idx += name_end

      # 列出剛剛找到的entity在文章中的資訊
      entity_info_list = [set(zip(start_idx, end_idx, NER_list, name_list))]

      review_PER = []
      for entity in sorted(entity_info_list[0]):
          # print(entity)
          if entity[2] == 'PERSON':
              review_PER.append(entity[3])

      Context_list=[]
      IBO_list = []

      sorted_entity_info_list = sorted(entity_info_list[0])

      for i, entity_info in enumerate(sorted_entity_info_list):
          # i = 0
          # j = sorted(entity_info_list[0])[i]
          # print(i, entity_info)
          word_start_loc, word_end_loc, flag, word = entity_info

          # 補O
          if word_start_loc != 0:
              if i != 0:
                  O_start_loc = sorted(sorted_entity_info_list)[i-1][1]
              else:
                  O_start_loc = 0

              O_end_loc = word_start_loc
              add_O_string = content[O_start_loc:O_end_loc]
              sub_split_words = split(add_O_string)
              sub_IBO = np.repeat('O', len(add_O_string)).tolist()

              Context_list += sub_split_words
              IBO_list += sub_IBO
          
          # 當圈人名
          sub_split_words = split(word)

          # 人名
          if (flag == 'PERSON') or (flag == 'ORG') or (flag == 'GPE'):

              # 人名
              if (flag == 'PERSON'):
                  flag2 = 'PER'
              # 機構名
              if (flag == 'ORG'):
                  flag2 = 'ORG'
              # 地名
              if (flag == 'GPE'):
                  flag2 = 'LOC'

              if len(word) > 1:
                  sub_IBO = ['B-' + flag2] + np.repeat('I-' + flag2, len(word) - 1).tolist()
              else:
                  sub_IBO = ['B-' + flag2]
          else:
              flag2 = 'O'
              sub_IBO = np.repeat(flag2, len(word)).tolist()
          
          Context_list += sub_split_words
          IBO_list += sub_IBO

          # 上面處理完，但最後一個人名後面還有一堆字需要補O
          if word_end_loc != len(content) and i == (len(sorted_entity_info_list)-1):
              word = content[word_end_loc:]
              sub_split_words = split(word)
              sub_IBO = np.repeat('O', len(word)).tolist()

              Context_list += sub_split_words
              IBO_list += sub_IBO

      Context_list.append('')
      IBO_list.append('')

    return Context_list, IBO_list


In [ ]:
context, ibo = BIO_encode(content=input_context, label=truth)

In [ ]:
def split_context_under_max_seqLen(input_content):
  split_new_content_list = []
  for content in input_content:
    if len(content)>=100:
      sub_split_new_content_list = [content[x:x+99] for x in range(0, len(content), 99)]
      for sub_split_content in sub_split_new_content_list:
          split_new_content_list += [sub_split_content]
    else:
      split_new_content_list += [content]
  return split_new_content_list

In [ ]:
def split(word):
    return [char for char in word]

# 整理BIO格式
all_content = []
all_BIO = []
fail_index =[]
wrong_index = []
all_page_idx = []

for k, url_info in news_urls.iterrows():
    input_content = str(url_info['content'])
    names = str(url_info['name']) # [] -> len=2
    
    # 先去掉人名為空 & 新聞爬不到的人 (0701改成全吃)
    if (input_content != 'nan') and (len(input_content) > 2): # (names != 'nan') and (len(names) > 2) and

        ground_truth = names.replace("[",'').replace("]",'').replace("'",'').split(",")
        Context_list, IBO_list = BIO_encode(content=input_content, label=ground_truth)
        
        # 用句號隔開
        sentence_period = (list(g) for _, g in groupby(Context_list, key='。'.__ne__))
        split_content_list = [a + b for a, b in zip(sentence_period, sentence_period)]
        
        # 整理成x, y
        ibo_iter = iter(IBO_list)
        split_IBO_list = [list(islice(ibo_iter, elem)) for elem in [len(i) for i in split_content_list]]
        
        
        # 如果x跟y的長度不一樣 -> 有問題
        if len([item for sublist in split_content_list for item in sublist]) != len([item for sublist in split_IBO_list for item in sublist]):
            wrong_index.append(k)
            print(f"index {k} has wrong match.")

        # (0702) 若句子長度>100, 則切斷, <100則維持原貌
        split_content_list = split_context_under_max_seqLen(input_content=split_content_list)
        split_IBO_list = split_context_under_max_seqLen(input_content=split_IBO_list)

        all_content += split_content_list
        all_BIO += split_IBO_list
        all_page_idx += [np.repeat(k, r).tolist() for r in map(len, split_content_list)]

        if (len(Context_list)-1) == len(input_content):
            # print(k, 'OK')
            pass
        else:
            print(k, 'FAIL')
            fail_index.append(k)
    else:
        # print(k, 'No Name', names)
        pass

In [ ]:
import pickle
with open('raw_data/all_content_emptyname_maxLen_0628.pkl', 'wb') as f:
  pickle.dump(all_content, f)
with open('raw_data/all_BIO_emptyname_maxLen_0628.pkl', 'wb') as f:
  pickle.dump(all_BIO, f)
with open('raw_data/all_idx_emptyname_maxLen_0628.pkl', 'wb') as f:
  pickle.dump(all_page_idx, f)